In [1]:
import pandas as pd
import time
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import missingno as msno
import sys


sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')


# 데이터 정제 시 필요한 모듈들
import src.subjectlist_alignment.pairing as pairing   # 삽관 발관 페어링 관련 모듈
import src.utils.utils as util   # 기타 유틸리티 모듈

# from src.subjectlist_alignment.impute_null import impute_intubation as ii
from src.subjectlist_alignment import reintubation as rnt


In [2]:
# 데이터 불러오기
paired_df = pd.read_csv('../outputs/paired_df.csv', index_col=0)

# 시간변수 변환
time_cols = ['intubationtime', 'extubationtime', 'admittime', 'dischtime', 'deathtime']
intubation_extubation = util.to_datetime(paired_df, time_cols)

print(f'intubation_extubation: {intubation_extubation.shape}')


Converted intubationtime to datetime.
Converted extubationtime to datetime.
Converted admittime to datetime.
Converted dischtime to datetime.
Converted deathtime to datetime.
intubation_extubation: (9396, 15)


In [34]:
paired_df[paired_df.intubationtime == paired_df.extubationtime]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker
1617,11745965,25740306,39859332.0,2119-04-04 01:08:00,2119-04-09 05:25:00,224385.0,277.0,39859332.0,2119-04-09 05:25:00,227194.0,277.0,Planned Extubation,2119-05-02 08:52:00,2119-05-02 08:52:00,NaN
2480,12683197,23772093,32225029.0,2145-07-31 01:07:00,2145-08-13 18:23:00,224385.0,74.0,32225029.0,2145-08-13 18:23:00,227194.0,74.0,Planned Extubation,2145-08-16 22:45:00,2145-08-16 22:45:00,NaN
3153,13358833,27550181,30867628.0,2136-10-26 19:36:00,2136-11-05 18:14:00,224385.0,83.4,30867628.0,2136-11-05 18:14:00,225468.0,83.4,Unplanned Extubation (patient-initiated),2136-11-10 20:45:00,2136-11-10 20:45:00,NaN
3398,13640324,24657887,32425117.0,2155-03-26 04:10:00,2155-03-28 13:48:00,224385.0,71.1,32425117.0,2155-03-28 13:48:00,225468.0,71.1,Unplanned Extubation (patient-initiated),2155-04-03 20:30:00,2155-04-03 20:30:00,NaN
4569,14896254,27841468,34403411.0,2165-07-10 10:00:00,2165-07-12 12:45:00,224385.0,92.7,34403411.0,2165-07-12 12:45:00,227194.0,92.7,Planned Extubation,2165-07-27 14:40:00,NaT,NaN


In [17]:
# def sort_ventilation_sequence(group):
#     """
#     Ventilation events (intubation and extubation) are sorted to establish a chronological order.
#     Rows with null intubationtime are sorted based on their extubationtime.
#     """

#     # Sort by intubationtime, with nulls at the end
#     group = group.sort_values(by=['intubationtime', 'extubationtime'], ascending=[True, True], na_position='last')

#     # Handle rows with null intubationtime
#     null_intubation_rows = group[group['intubationtime'].isnull()]
#     for idx, row in null_intubation_rows.iterrows():
#         # Find a suitable location based on extubationtime
#         suitable_location = group[group['extubationtime'] <= row['extubationtime']].last_valid_index()
#         if suitable_location:
#             # Reorder the row to its suitable location
#             group = pd.concat([group.loc[:suitable_location], pd.DataFrame([row]), group.loc[suitable_location+1:]]).reset_index(drop=True)
#         else:
#             # If no suitable location, place at the beginning
#             group = pd.concat([pd.DataFrame([row]), group]).reset_index(drop=True)

#     return group

# def sort_ventilation_sequence(group):
#     """
#     Ventilation events (intubation and extubation) are sorted to establish a chronological order.
#     Rows with null intubationtime are sorted based on their extubationtime.
#     """

#     # Initially sort by intubationtime and extubationtime, with nulls at the end
#     group = group.sort_values(by=['intubationtime', 'extubationtime'], ascending=[True, True], na_position='last')

#     # Handle rows with null intubationtime
#     null_intubation_indices = group[group['intubationtime'].isnull()].index
#     for idx in null_intubation_indices:
#         row = group.loc[idx]
#         # Find a suitable location based on extubationtime
#         suitable_location = group[group['extubationtime'] <= row['extubationtime']].last_valid_index()
#         # Drop the original row
#         group = group.drop(idx)
#         # Reorder the row to its suitable location
#         if suitable_location and suitable_location != idx:
#             group = pd.concat([group.loc[:suitable_location], pd.DataFrame([row]), group.loc[suitable_location+1:]]).reset_index(drop=True)
#         elif suitable_location is None:
#             # If no suitable location, place at the beginning
#             group = pd.concat([pd.DataFrame([row]), group]).reset_index(drop=True)

#     return group

def sort_ventilation_sequence(group):
    """
    삽관/발관 이벤트를 시간 순서에 맞게 재정렬합니다(null 값 고려). 
    단계:
    1. 삽관시간 -> 발관시간 순으로 정렬 (이때, 삽관시간이 null이면 순서가 틀어짐.)
    2. 삽관시간이 null인 경우는 따로 빼놓고, 삽관시간이 not-null인 경우 먼저 정렬(단순 정렬)
    3. 삽관시간이 null인 행의 발관시간을 다른 행들의 발관시간과 비교
    4. 적절한 발관시간 위치에 해당 행 인서트

    """

    # 삽관시간/발관시간 순으로 정렬
    sorted_group = group.sort_values(by=['intubationtime', 'extubationtime'], ascending=[True, True], na_position='last')

    # 새로운 데이터프레임 생성
    sorted_df = pd.DataFrame(columns=group.columns)

    # intubationtime이 null인 경우 따로 빼놓기
    null_intubation_rows = sorted_group[sorted_group['intubationtime'].isnull()]

    # intubationtime이 결측치가 아닌 경우 먼저 처리
    sorted_df = pd.concat([sorted_df, sorted_group[~sorted_group.index.isin(null_intubation_rows.index)]])

    # intubationtime 결측치인 경우, extubationtime을 기준으로 해당 행의 위치 찾아주기
    for idx, row in null_intubation_rows.iterrows():
        # extubationtime의 순서에 맞게 행 넣어주기
        suitable_location = sorted_df[sorted_df['extubationtime'] <= row['extubationtime']].last_valid_index()
        if suitable_location is not None:
            sorted_df = pd.concat([sorted_df.loc[:suitable_location], pd.DataFrame([row]), sorted_df.loc[suitable_location+1:]]).reset_index(drop=True)
        else:
            # 적절한 위치가 없을 경우, 맨 앞에 넣어줌.
            sorted_df = pd.concat([pd.DataFrame([row]), sorted_df]).reset_index(drop=True)

    return sorted_df


def carryover_next_intubationtime(group):
    """
    다음행의 intubationtime을 현재행의 reintubation_eventtime 칼럼으로 가져옵니다.
    """
    if len(group) > 1:
        # 마지막 행을 제외하고 순회
        for i in range(len(group) - 1):
            current_row = group.iloc[i]
            next_row = group.iloc[i + 1]

            # 다음행의 intubationtime을 현재 행의 reintubation_eventtime으로 가져옴
            # 다음행의 intubationtime이 null일 경우, reintubation_eventtime도 null.
            group.at[current_row.name, 'reintubation_eventtime'] = next_row['intubationtime']

    return group


def get_reintubationtime(group):
    """
    'reintubation_eventtime' (다음 intubationtime)과 현재의 'extubationtime'의 시간차를 구합니다 (분 단위; float).
    """
    for idx, row in group.iterrows():
        # Not-null 값인지 확인
        if pd.notna(row['reintubation_eventtime']) and pd.notna(row['extubationtime']):
            # 시간차 계산 (분)
            time_diff = (row['reintubation_eventtime'] - row['extubationtime']).total_seconds() / 60
            group.at[idx, 'reintubationtime'] = time_diff
        else:
            # 어느 한쪽이 null이라 계산이 안될 경우, None 입력
            group.at[idx, 'reintubationtime'] = None

    return group



In [22]:
def get_reintubation(df):
    grouped_df = df.groupby(['subject_id', 'hadm_id'])

    df_list = []
    for _, patient_df in grouped_df:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            
            patient_df = rnt.create_reintubation_columns(patient_df)   # reintubationtime 칼럼 초기화
            patient_df = rnt.sort_ventilation_sequence(patient_df)  
            patient_df = rnt.carryover_next_intubationtime(patient_df) 
            patient_df = rnt.get_reintubationtime(patient_df) 
        
        df_list.append(patient_df)

    # # 환자별 데이터 하나의 데이터프레임으로 합치기
    # reintubation_df = pd.concat(df_list)
        
    return df_list

In [4]:
test_subs = list(paired_df[paired_df.intubationtime.isna()].hadm_id.unique())

test_df = paired_df[paired_df.hadm_id.isin(test_subs)]
test_df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker
14,10020740,23831430,35889503.0,2150-03-11 15:34:00,2150-03-27 07:44:00,224385.0,103.0,35889503.0,2150-03-27 16:33:00,227194.0,103.0,Planned Extubation,2150-04-25 13:50:00,NaT,NaN
15,10020740,23831430,31077365.0,2150-03-11 15:34:00,2150-03-30 08:20:00,224385.0,103.0,31077365.0,2150-04-02 01:09:00,227194.0,103.0,Planned Extubation,2150-04-25 13:50:00,NaT,NaN
16,10020740,23831430,NaN,2150-03-11 15:34:00,NaT,NaN,NaN,35026312.0,2150-03-12 18:15:00,227194.0,102.5,Planned Extubation,2150-04-25 13:50:00,NaT,NaN
17,10020740,23831430,NaN,2150-03-11 15:34:00,NaT,NaN,NaN,35044342.0,2150-03-19 11:05:00,227194.0,103.0,Planned Extubation,2150-04-25 13:50:00,NaT,NaN
21,10024982,25154057,37919901.0,2203-09-19 12:29:00,2203-10-02 11:00:00,224385.0,79.1,37919901.0,2203-10-09 14:50:00,227194.0,79.1,Planned Extubation,2203-10-11 19:10:00,2203-10-11 19:10:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9387,19995595,21784060,34670930.0,2126-10-17 21:43:00,2126-10-31 00:45:00,224385.0,94.7,34670930.0,2126-11-03 16:20:00,227194.0,94.7,Planned Extubation,2126-11-12 17:35:00,NaT,NaN
9388,19995595,21784060,NaN,2126-10-17 21:43:00,NaT,NaN,NaN,34670930.0,2126-10-30 14:26:00,227194.0,94.7,Planned Extubation,2126-11-12 17:35:00,NaT,NaN
9390,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN


In [5]:
test_df[test_df.hadm_id == 28166872]



,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker
24,10027602,28166872,32391858.0,2201-10-30 12:05:00,2201-11-07 15:00:00,224385.0,64.0,32391858.0,2201-11-09 12:45:00,227194.0,64.0,Planned Extubation,2201-11-20 14:45:00,NaT,NaN
25,10027602,28166872,NaN,2201-10-30 12:05:00,NaT,NaN,NaN,32391858.0,2201-11-07 13:53:00,227194.0,64.0,Planned Extubation,2201-11-20 14:45:00,NaT,NaN


In [6]:
df_list = get_reintubation(test_df)
len(df_list)



/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철/src/subjectlist_alignment/reintubation.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_df = pd.concat([sorted_df, sorted_group[~sorted_group.index.isin(null_intubation_rows.index)]])
/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철/src/subjectlist_alignment/reintubation.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_df = pd.concat([sorted_df, sorted_group[~sorted_group.index.isin(null_intubation_rows.inde

1043

In [7]:
df_list[1010]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,reintubation_eventtime,reintubationtime
0,19686959,29276030,35805323.0,2139-05-24 17:30:00,2139-05-24 20:00:00,224385.0,118.0,35805323.0,2139-05-25 08:01:00,227194.0,118.0,Planned Extubation,2139-06-23 17:00:00,NaT,NaN,NaT,NaN
1,19686959,29276030,NaN,2139-05-24 17:30:00,NaT,NaN,NaN,37747740.0,2139-05-30 10:30:00,227194.0,118.0,Planned Extubation,2139-06-23 17:00:00,NaT,NaN,2139-06-13 15:30:00,20460.0
2,19686959,29276030,37532348.0,2139-05-24 17:30:00,2139-06-13 15:30:00,224385.0,111.4,37532348.0,2139-06-16 19:00:00,227194.0,111.4,Planned Extubation,2139-06-23 17:00:00,NaT,NaN,NaN,NaN


### 전체 데이터로 테스트

In [23]:
df_list_full = get_reintubation(paired_df)
len(df_list_full)
concatenated_df = pd.concat(df_list_full, ignore_index=True)

/var/folders/1m/_xkztmr10ks17qr1v6jbfngr0000gn/T/ipykernel_95456/2910570689.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concatenated_df = pd.concat(df_list_full, ignore_index=True)


In [19]:
grouped_final = paired_df.groupby(['subject_id', 'hadm_id'])

multi_row_groups = []
for (subject, hadm), g in grouped_final:
    if len(g) > 1:
        multi_row_groups.append((subject, hadm))


In [21]:
multi_row_groups[100]

(10641795, 22739609)

In [27]:
concatenated_df[concatenated_df.hadm_id == 22739609][['intubationtime', 'extubationtime', 'reintubation_eventtime', 'reintubationtime']]

,intubationtime,extubationtime,reintubation_eventtime,reintubationtime
577,2146-04-28 08:20:00,2146-04-29 12:05:00,2146-04-29 12:35:00,30.0
578,2146-04-29 12:35:00,2146-05-06 14:00:00,NaN,NaN


In [28]:
# Convert the list of tuples to a DataFrame for easy filtering
multi_row_filter = pd.DataFrame(multi_row_groups, columns=['subject_id', 'hadm_id'])

# Filter concatenated_df to include only rows that match the (subject_id, hadm_id) pairs in multi_row_filter
filtered_df = concatenated_df[concatenated_df.set_index(['subject_id', 'hadm_id']).index.isin(multi_row_filter.set_index(['subject_id', 'hadm_id']).index)]

# Reset the index of the filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)


In [33]:
reint_test_df = filtered_df[['subject_id', 'hadm_id', 'intubationtime', 'extubationtime', 'reintubation_eventtime', 'reintubationtime']]
reint_test_df.to_csv('../outputs/reint_test_df.csv')